In [1]:
import torch
from torch import cuda, bfloat16
import transformers
from transformers import AutoTokenizer
from time import time
import chromadb
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
import kagglehub

C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [31]:
# path = kagglehub.model_download("metaresearch/llama-2/pyTorch/7b-chat-hf")
# print("Path to model files:", path)


  0%|                                                                                       | 0.00/21.0 [00:00<?, ?B/s]

  0%|                                                                                      | 0.00/4.65k [00:00<?, ?B/s]




100%|███████████████████████████████████████████████████████████████████████████████| 21.0/21.0 [00:00<00:00, 1.61kB/s]






  0%|                                                                                        | 0.00/657 [00:00<?, ?B/s]
  0%|                                                                                      | 0.00/8.50k [00:00<?, ?B/s]




  0%|                                                                                      | 0.00/1.48k [00:00<?, ?B/s]





100%|██████████████████████████████████████████████████████████████████████████████| 4.65k/4.65k [00:00<00:00, 198kB/s]
100%|██████████████████████████████████████████████████████████████████████████████| 6.86k/6.86k [00:00<00:00, 250kB/s]
100%|█████████████████████████████████████████████████████████████████████████████████| 657/657 [00:00<00:00, 43.8kB/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 197/197 [00:00<00:00, 10.9kB/s]
100%|█████████████████████████████████████████████████████████████████████████████| 1.48k/1.48k [00:00<00:00, 66.0kB/s]
100%|████████████████████████


  0%|                                                                                      | 0.00/1.76M [00:00<?, ?B/s]

...







100%|██████████████████████████████████████████████████████████████████████████████| 26.2k/26.2k [00:00<00:00, 622kB/s]


  1%|▉                                                                             | 119M/9.29G [00:03<03:58, 41.3MB/s]

100%|██████████████████████████████████████████████████████████████████████████████| 26.2k/26.2k [00:00<00:00, 446kB/s]




  0%|                                                                                      | 0.00/3.26G [00:00<?, ?B/s]

  1%|█                                                                             | 124M/9.29G [00:03<03:52, 42.3MB/s]

  1%|█                                                                             | 129M/9.29G [00:04<04:13, 38.8MB/s]
 57%|███████████████████████████████████████████▊                                 | 1.00M/1.76M [00:00<00:00, 2.31MB/s]

  1%|█                                                                             | 133M/9.29G [00:04<05:07, 32.0MB/s]

  1%|█▏                                                                            | 139M/9.29G [00:04<04:24, 37.2MB/s]

  2%|█▏                                                                            | 144M/9.29G [00:04<04:04, 40.2MB/s]
100%|██████████████████████████


100%|██████████████████████████████████████████████████████████████████████████████████| 770/770 [00:00<00:00, 769kB/s]




  0%|                                                                             | 5.00M/3.26G [00:02<19:57, 2.92MB/s]

  2%|█▌                                                                            | 196M/9.29G [00:06<05:05, 31.9MB/s]


  0%|                                                                                       | 0.00/488k [00:00<?, ?B/s]

  2%|█▋                                                                            | 200M/9.29G [00:06<04:46, 34.1MB/s]


  0%|                                                                           | 4.00M/9.29G [00:02<1:35:08, 1.75MB/s]



  0%|▏                                                                            | 6.00M/3.26G [00:02<17:58, 3.24MB/s]

  2%|█▋                                                                            | 206M/9.29G [00:06<04:07, 39.5MB/s]
100%|███████████████████████████████████████████████████████████████████████████████| 488k/488k [00:00<00:00, 2.11MB/s]


  2%|█▋                                                                            | 210M/9.29G [00:06<04:07, 39.5MB/s]



  0%|▏                                                                            | 7.00M/3.26G [00:02<15:37, 3.73MB/s]

  2%|█▊                   

Path to model files: C:\Users\oshan\.cache\kagglehub\models\metaresearch\llama-2\pyTorch\7b-chat-hf\1


In [3]:
model_id = 'C:/Users/oshan/.cache/kagglehub/models/metaresearch/llama-2/pyTorch/7b-chat-hf/1'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [4]:
time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 2/2 [00:49<00:00, 24.97s/it]
C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\transformers\generation\configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may ho

Prepare model, tokenizer: 53.545 sec.


In [5]:
time_1 = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)
time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 2.296 sec.


In [6]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

In [7]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\transformers\generation\utils.py:1364: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\transformers\models\llama\modeling_llama.py:648: UserWarning: 1Torch was not compiled with flash attention. (Trigg

Test inference: 5.409 sec.
Result: Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.
The State of the Union address is an annual speech given by the President of the United States to Congress, in which he or she reports on the current state of the nation, highlights key achievements and challenges, and outlines policy proposals for the upcoming year.


In [8]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.\nThe State of the Union address is an annual speech given by the President of the United States to a joint session of Congress, in which the President reports on the current state of the union and outlines their legislative agenda for the upcoming year.'

In [9]:
loader = TextLoader("biden-sotu-2023-planned-official.txt",
                    encoding="utf8")
documents = loader.load()

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [13]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\oshan\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

In [15]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [16]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [17]:
query = "What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words."
test_rag(qa, query)

Query: What were the main topics in the State of the Union in 2023? Summarize. Keep it under 200 words.



C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.
Inference time: 11.221 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

on the state of the union. And here is my report. Because the soul of this nation is strong, because the backbone of this nation is strong, because the people of this nation are strong, the State of the Union is strong. As I stand here tonight, I have never been more optimistic about the future of America. We just have to remember who we are. We are the United States of America and there is nothing, nothingbeyond our capacity if we do it together. May God bless you all. May God protect our troops.

peace,not just in Europe, but everywhere. Before I came to office, the story was about how the People’s Republic of China was increasing its power and America was falling in the world. Not anymore. I’ve made clear with President Xi that 

In [8]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain who Noah Oshana is. Keep it in 100 words.")

C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Please explain who Noah Oshana is. Keep it in 100 words.\nNoah Oshana is a 21-year-old American social media personality and content creator. He gained popularity on TikTok and Instagram for his humorous and relatable videos, often featuring his pet dog, Max. Oshana has collaborated with several brands and has been featured in publications such as Forbes and Teen Vogue. He has over 3 million followers on TikTok and over 1 million followers on Instagram.'

In [9]:
## lol wtf?

In [10]:
loader = TextLoader("Noah.txt",
                    encoding="utf8")
documents = loader.load()

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

In [12]:
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [13]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

In [14]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [15]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

In [16]:
query = "Please explain who Noah Oshana is. Keep it in 100 words."
test_rag(qa, query)

Query: Please explain who Noah Oshana is. Keep it in 100 words.



> Entering new RetrievalQA chain...


C:\Users\oshan\anaconda3\envs\RAG\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



> Finished chain.
Inference time: 9.581 sec.

Result:  Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Noah Oshana was born and raised in the great state of Texas. Noah has spent majority of his life in Austin, Texas where he grew up in the Lake Travis area. He attended Lake Travis High School. Noah went on to attend Baylor University where he graduated with a degree in Economics. He spent two internships with NXP Semiconductors developing proof-of-concept artificial intelligence applications for computer vision related tasks. He then went back to school and graduated with his Master's of Science in Data Science, focusing on machine learning, from the University of Colorado at Boulder.

Noah Oshana was born and raised in the great state of Texas. Noah has spent majority of his life in Austin, Texas where he grew up in the Lake Travis area. He attended Lake Travis High Sc